<a id="plan"></a>

## Data Preprocessing

-  [About the dataset and Kaggle](#about)<br>
-  [Load the data](#load_data)<br>
-  [Look at the data](#look_data)<br>
-  [Look at the columns](#look_columns)<br>
-  [Variable to predict](#output_variable)<br>
-  [Naive first model training](#naive_training)<br>
-  [Convert date times](#convert_date)<br>
-  [Convert strings into numerotated categories](#convert_strings)<br>
-  [Inspect missing values](#nans)<br>
-  [Save preprocessed data](#save_data)<br>
-  [Fully numericalize data](#numericalize_data)<br>



### Imports

In [7]:
# for package auto reload
%load_ext autoreload
%autoreload 2

# for better rendering of plots in jupyter notebook
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# base modules
import os
import sys
import copy


# for manipulating data
import numpy as np
import pandas as pd



In [3]:
# path to repo
# path_to_repo = os.path.dirname(os.getcwd())
path_to_repo = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
path_to_repo

'/Users/sugumaran/Documents/EM-LYON/Advanced Machine Learning Models/Day 1/EMLyon-ML-Course-2022-T1-main'

In [16]:
sys.path.insert(0, os.path.join(path_to_repo, 'src'))

# custom module
from emlyon.utils import *

In [17]:
path_to_bulldozers = os.path.join(path_to_repo, "data", "bulldozers", "")
path_to_bulldozers

'/Users/sugumaran/Documents/EM-LYON/Advanced Machine Learning Models/Day 1/EMLyon-ML-Course-2022-T1-main/data/bulldozers/'

# Data preprocessing

[Back to top](#plan)

<a id="about"></a>

### $\bullet$ About the dataset and Kaggle

[Back to top](#plan)

We will be looking at the Blue Book for Bulldozers Kaggle Competition: "The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuration.  The data is sourced from auction result postings and includes information on usage and equipment configurations."

This is a very common type of dataset and prediciton problem, and similar to what you may see in your project or workplace.

Kaggle is an awesome resource for aspiring data scientists or anyone looking to improve their machine learning skills.  There is nothing like being able to get hands-on practice and receiving real-time feedback to help you improve your skills.

Kaggle provides:

1. Interesting data sets
2. Feedback on how you're doing
3. A leader board to see what's good, what's possible, and what's state-of-art.
4. Blog posts by winning contestants share useful tips and techniques.



Kaggle provides info about some of the fields of our dataset; on the [Kaggle Data info](https://www.kaggle.com/c/bluebook-for-bulldozers/data) page they say the following:

For this competition, you are predicting the sale price of bulldozers sold at auctions. The data for this competition is split into three parts:

- **Train.csv** is the training set, which contains data through the end of 2011.
- **Valid.csv** is the validation set, which contains data from January 1, 2012 - April 30, 2012. You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
- **Test.csv** is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

<a id="load_data"></a>

### $\bullet$ Import the data

[Back to top](#plan)

In [18]:
# chose one of the equivalent 
path_to_datasets = os.path.join(path_to_bulldozers, 'Train.csv')
print(path_to_datasets)

path_to_datasets = '{}Train.csv'.format(path_to_bulldozers)
print(path_to_datasets)

/Users/sugumaran/Documents/EM-LYON/Advanced Machine Learning Models/Day 1/EMLyon-ML-Course-2022-T1-main/data/bulldozers/Train.csv
/Users/sugumaran/Documents/EM-LYON/Advanced Machine Learning Models/Day 1/EMLyon-ML-Course-2022-T1-main/data/bulldozers/Train.csv


In [20]:
df_raw = pd.read_csv(path_to_datasets, low_memory = False, parse_dates = ["saledate"])

<a id="look_data"></a>

### $\bullet$ Look at the data

[Back to top](#plan)

In any sort of data science work, it's **important to look at your data**, to make sure you understand the format, how it's stored, what type of values it holds, etc.<br> 
Even if you've read descriptions about your data, the actual data may not be what you expect.

The key fields that are in train.csv are:

- SalesID: the unique identifier of the sale
- MachineID: the unique identifier of a machine.  A machine can be sold multiple times
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate: the date of the sale

In [21]:
# print the 5 first and last rows
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [22]:
# print the first n rows (default is n = 5)
#df_raw.head(n = 3)
df_raw.head(3)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
help(df_raw.head)

Help on method head in module pandas.core.generic:

head(n: 'int' = 5) -> 'NDFrameT' method of pandas.core.frame.DataFrame instance
    Return the first `n` rows.
    
    This function returns the first `n` rows for the object based
    on position. It is useful for quickly testing if your object
    has the right type of data in it.
    
    For negative values of `n`, this function returns all rows except
    the last `|n|` rows, equivalent to ``df[:n]``.
    
    If n is larger than the number of rows, this function returns all rows.
    
    Parameters
    ----------
    n : int, default 5
        Number of rows to select.
    
    Returns
    -------
    same type as caller
        The first `n` rows of the caller object.
    
    See Also
    --------
    DataFrame.tail: Returns the last `n` rows.
    
    Examples
    --------
    >>> df = pd.DataFrame({'animal': ['alligator', 'bee', 'falcon', 'lion',
    ...                    'monkey', 'parrot', 'shark', 'whale', 'zebra']})
 

<a id="look_columns"></a>

### $\bullet$ Look at the columns

[Back to top](#plan)

In [25]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [30]:
df_raw.dtypes

SalesID                              int64
SalePrice                            int64
MachineID                            int64
ModelID                              int64
datasource                           int64
auctioneerID                       float64
YearMade                             int64
MachineHoursCurrentMeter           float64
UsageBand                           object
saledate                    datetime64[ns]
fiModelDesc                         object
fiBaseModel                         object
fiSecondaryDesc                     object
fiModelSeries                       object
fiModelDescriptor                   object
ProductSize                         object
fiProductClassDesc                  object
state                               object
ProductGroup                        object
ProductGroupDesc                    object
Drive_System                        object
Enclosure                           object
Forks                               object
Pad_Type   

In [26]:
len(df_raw.columns)

53

In [27]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [28]:
df_raw.shape

(401125, 53)

In [29]:
display_all(df_raw.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3.0,3.0,3.0,3.0,3.0
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68.0,4640.0,2838.0,3486.0,722.0
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [31]:
display_all(df_raw.describe(include = 'all').T)

/var/folders/3c/hjtyk_rn3jg_lml1bd3q8fbc0000gn/T/ipykernel_6319/956724908.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display_all(df_raw.describe(include = 'all').T)


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125.0,NaN,NaN,NaN,NaT,NaT,1919712.521503,909021.492667,1139246.0,1418371.0,1639422.0,2242707.0,6333342.0
SalePrice,401125.0,NaN,NaN,NaN,NaT,NaT,31099.712848,23036.898502,4750.0,14500.0,24000.0,40000.0,142000.0
MachineID,401125.0,NaN,NaN,NaN,NaT,NaT,1217902.517971,440991.954249,0.0,1088697.0,1279490.0,1468067.0,2486330.0
ModelID,401125.0,NaN,NaN,NaN,NaT,NaT,6889.70298,6221.777842,28.0,3259.0,4604.0,8724.0,37198.0
datasource,401125.0,NaN,NaN,NaN,NaT,NaT,134.66581,8.962237,121.0,132.0,132.0,136.0,172.0
auctioneerID,380989.0,NaN,NaN,NaN,NaT,NaT,6.55604,16.976779,0.0,1.0,2.0,4.0,99.0
YearMade,401125.0,NaN,NaN,NaN,NaT,NaT,1899.156901,291.797469,1000.0,1985.0,1995.0,2000.0,2013.0
MachineHoursCurrentMeter,142765.0,NaN,NaN,NaN,NaT,NaT,3457.955353,27590.256413,0.0,0.0,0.0,3025.0,2483300.0
UsageBand,69639,3,Medium,33985,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17,2011-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id="output_variable"></a>

### $\bullet$ Variable to predict

[Back to top](#plan)

The variable we want to predict is the sale price of a bulldozer, based on its description. This means that we seek to **predict** values within the _SalePrice_ column of the dataset, by using the other columns as **expainatory variables** for prediction.

It's important to note what metric is being used for a project. Generally, selecting the metric(s) is an important part of the project setup. However, in this case Kaggle tells us what metric to use: RMSLE (root mean squared log error) between the actual and predicted auction prices. Therefore we take the log of the prices, so that RMSE will give us what we need.

In [32]:
df_raw.SalePrice

0         66000
1         57000
2         10000
3         38500
4         11000
          ...  
401120    10500
401121    11000
401122    11500
401123     9000
401124     7750
Name: SalePrice, Length: 401125, dtype: int64

In [33]:
sale_prices = df_raw.SalePrice.tolist()

In [34]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [35]:
df_raw.SalePrice

0         11.097410
1         10.950807
2          9.210340
3         10.558414
4          9.305651
            ...    
401120     9.259131
401121     9.305651
401122     9.350102
401123     9.104980
401124     8.955448
Name: SalePrice, Length: 401125, dtype: float64

In [36]:
display_all(df_raw.describe(include = 'all').T)

/var/folders/3c/hjtyk_rn3jg_lml1bd3q8fbc0000gn/T/ipykernel_6319/956724908.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display_all(df_raw.describe(include = 'all').T)


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125.0,NaN,NaN,NaN,NaT,NaT,1919712.521503,909021.492667,1139246.0,1418371.0,1639422.0,2242707.0,6333342.0
SalePrice,401125.0,NaN,NaN,NaN,NaT,NaT,10.103096,0.693621,8.4659,9.581904,10.085809,10.596635,11.863582
MachineID,401125.0,NaN,NaN,NaN,NaT,NaT,1217902.517971,440991.954249,0.0,1088697.0,1279490.0,1468067.0,2486330.0
ModelID,401125.0,NaN,NaN,NaN,NaT,NaT,6889.70298,6221.777842,28.0,3259.0,4604.0,8724.0,37198.0
datasource,401125.0,NaN,NaN,NaN,NaT,NaT,134.66581,8.962237,121.0,132.0,132.0,136.0,172.0
auctioneerID,380989.0,NaN,NaN,NaN,NaT,NaT,6.55604,16.976779,0.0,1.0,2.0,4.0,99.0
YearMade,401125.0,NaN,NaN,NaN,NaT,NaT,1899.156901,291.797469,1000.0,1985.0,1995.0,2000.0,2013.0
MachineHoursCurrentMeter,142765.0,NaN,NaN,NaN,NaT,NaT,3457.955353,27590.256413,0.0,0.0,0.0,3025.0,2483300.0
UsageBand,69639,3,Medium,33985,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17,2011-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# same as 
# df_raw.SalePrice
df_raw['SalePrice']

0         11.097410
1         10.950807
2          9.210340
3         10.558414
4          9.305651
            ...    
401120     9.259131
401121     9.305651
401122     9.350102
401123     9.104980
401124     8.955448
Name: SalePrice, Length: 401125, dtype: float64

In [38]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [39]:
# to convert back prices to original values
np.exp(df_raw.SalePrice)

0         66000.0
1         57000.0
2         10000.0
3         38500.0
4         11000.0
           ...   
401120    10500.0
401121    11000.0
401122    11500.0
401123     9000.0
401124     7750.0
Name: SalePrice, Length: 401125, dtype: float64

<a id="naive_training"></a>

### $\bullet$ Naive first model training

[Back to top](#plan)

In [40]:
from sklearn.ensemble import RandomForestRegressor

In [41]:
m = RandomForestRegressor(n_jobs = -1)

In [42]:
# The following code is supposed to fail due to string values in the input data
m.fit(df_raw.drop('SalePrice', axis = 1), df_raw.SalePrice)

ValueError: could not convert string to float: 'Low'

This dataset contains a mix of **continuous** and **categorical** variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals.  You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

<a id="convert_date"></a>

### $\bullet$ Convert date times

[Back to top](#plan)

In [43]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [44]:
df_raw.fiModelDesc.value_counts()

310G        5039
416C        4869
580K        4315
310E        4233
140G        4083
            ... 
EX210-5        1
KX025          1
EX120-5F       1
EX100-5E       1
HW180          1
Name: fiModelDesc, Length: 4999, dtype: int64

In [45]:
df_raw.saledate

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

In [46]:
fld = df_raw.saledate

In [47]:
fld

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

In [48]:
# date-time object, was created within the df with parse_dates=["saledate"]
fld.dt

In [49]:
help(add_datepart)

Help on function add_datepart in module emlyon.utils:

add_datepart(df, fldnames, drop=True, time=False, errors='raise')
    add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string or list of strings that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek

In [50]:
df_backup = copy.deepcopy(df_raw)

In [51]:
df_backup2 = df_raw.copy()

In [52]:
df_backup3 = df_raw

In [53]:
# add_datepart adds new columns to the dataframe, INPLACE
add_datepart(df_raw, 'saledate')

/Users/sugumaran/Documents/EM-LYON/Advanced Machine Learning Models/Day 1/EMLyon-ML-Course-2022-T1-main/src/emlyon/utils.py:161: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())


In [54]:
# comparaison of dataframe shape before & after applying add_datepart
print(df_backup.shape)
print(df_backup2.shape)
print(df_backup3.shape)
print(df_raw.shape)

(401125, 53)
(401125, 53)
(401125, 65)
(401125, 65)


In [55]:
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [56]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [57]:
df_raw.saleDayofweek

0         3
1         4
2         3
3         3
4         3
         ..
401120    2
401121    2
401122    2
401123    1
401124    1
Name: saleDayofweek, Length: 401125, dtype: int64

In [58]:
len(df_raw.columns)

65

<a id="convert_strings"></a>

### $\bullet$ Convert strings into numerotated categories

[Back to top](#plan)

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call `train_cats` to convert strings to pandas categories.

In [129]:
help(train_cats)

Help on function train_cats in module emlyon.utils:

train_cats(df)
    Change any columns of strings in a panda's dataframe to a column of
    categorical values. This applies the changes inplace.
    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category



In [130]:
train_cats(df_raw)

We can specify the order to use for categorical variables if we wish:

In [132]:
df_backup.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

In [131]:
df_raw.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

In [133]:
df_raw.UsageBand.cat.categories.tolist()

['High', 'Low', 'Medium']

In [134]:
df_raw.UsageBand.cat.set_categories(['Low', 'Medium', 'High'], ordered = True, inplace = True)

C:\Users\Jb\anaconda3\envs\emlyon\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [135]:
df_raw.UsageBand.cat.categories.tolist()

['Low', 'Medium', 'High']

Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.

In [136]:
df_raw.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

In [137]:
df_raw.UsageBand.cat.categories

Index(['Low', 'Medium', 'High'], dtype='object')

In [138]:
df_raw.UsageBand.cat.codes

0         0
1         0
2         2
3         2
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [139]:
df.drop_duplicates(subset=['brand'])

In [140]:
# - values corresponds to NaN values in the original column
df_raw.UsageBand

0         0
1         0
2         2
3         2
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Name: UsageBand, Length: 401125, dtype: int8

<a id="nans"></a>

### $\bullet$ Inspect missing values

[Back to top](#plan)

We're still not quite done - for instance we have lots of missing values, which we can't pass directly to a random forest.

In [141]:
df_raw.isnull().describe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
count,401125,401125,401125,401125,401125,401125,401125,401125,401125,401125,...,401125,401125,401125,401125,401125,401125,401125,401125,401125,401125
unique,1,1,1,1,1,2,1,2,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,401125,401125,401125,401125,401125,380989,401125,258360,401125,401125,...,401125,401125,401125,401125,401125,401125,401125,401125,401125,401125


In [142]:
display_all(df_raw.isnull().describe().T)

,count,unique,top,freq
SalesID,401125,1,False,401125
SalePrice,401125,1,False,401125
MachineID,401125,1,False,401125
ModelID,401125,1,False,401125
datasource,401125,1,False,401125
auctioneerID,401125,2,False,380989
YearMade,401125,1,False,401125
MachineHoursCurrentMeter,401125,2,True,258360
UsageBand,401125,1,False,401125
fiModelDesc,401125,1,False,401125


In [143]:
# display amount of missing values per column :
# column name              ratio of nans
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

<a id="save_data"></a>

### $\bullet$ Save preprocessed data

[Back to top](#plan)

But let's save this file for now, since it's already in format can we be stored and accessed efficiently.

In [144]:
path_to_tmp = os.path.join(path_to_repo, "data", "bulldozers")

In [145]:
path_to_tmp

'C:\\Users\\Jb\\Desktop\\NLP\\Teaching - EMLyon ML Course\\2022 T1\\data\\bulldozers'

In [146]:
os.makedirs(path_to_tmp, exist_ok = True)

In [147]:
path_to_bulldozers_raw = os.path.join(path_to_tmp, 'bulldozers-raw')

In [148]:
path_to_bulldozers_raw

'C:\\Users\\Jb\\Desktop\\NLP\\Teaching - EMLyon ML Course\\2022 T1\\data\\bulldozers\\bulldozers-raw'

In [63]:
# 18 MB
df_raw.to_feather(path_to_bulldozers_raw)

In [64]:
# 140 MB !!
df_raw.to_csv(path_to_bulldozers_raw + '.csv', index = False)

<a id="numericalize_data"></a>

### $\bullet$ Fully numericalize data

[Back to top](#plan)

In the future we can simply read it from this fast format.

In [151]:
path_to_bulldozers_raw = os.path.join(path_to_repo, 'data', 'bulldozers', 'bulldozers-raw')

df_raw = pd.read_feather(path_to_bulldozers_raw)
# or
#df_raw = pd.read_csv(path_to_bulldozers_raw + '.csv', low_memory = False)

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [152]:
help(proc_df)

Help on function proc_df in module emlyon.utils:

proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None, preproc_fn=None, max_n_cat=None, subset=None, mapper=None)
    proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe. For each column of df 
    which is not in skip_flds nor in ignore_flds, na values are replaced by the
    median value of the column.
    Parameters:
    -----------
    df: The data frame you wish to process.
    y_fld: The name of the response variable
    skip_flds: A list of fields that dropped from df.
    ignore_flds: A list of fields that are ignored during processing.
    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)
    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.
    preproc_fn: A function that gets applied to df.
    max_n_cat: The maximum number of categorie

In [153]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [154]:
df.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'fiModelDesc',
       'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
       'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type',
       'Ride_Control', 'Stick', 'Transmission', 'Turbocharged',
       'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower',
       'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control',
       'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks',
       'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width',
       'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type',
       'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',
       'saleI

In [155]:
df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,999089,3157,121,3.0,2004,68.0,0,950,296,...,320,False,False,False,False,False,False,1163635200,False,False
1,1139248,117657,77,121,3.0,1996,4640.0,0,1725,527,...,86,False,False,False,False,False,False,1080259200,False,False
2,1139249,434808,7009,121,3.0,2001,2838.0,2,331,110,...,57,False,False,False,False,False,False,1077753600,False,False
3,1139251,1026470,332,121,3.0,2001,3486.0,2,3674,1375,...,139,False,False,False,False,False,False,1305763200,False,False
4,1139253,1057373,17311,121,3.0,2007,722.0,1,4208,1529,...,204,False,False,False,False,False,False,1248307200,False,False


In [156]:
y

array([11.09741002, 10.95080655,  9.21034037, ...,  9.35010231,
        9.10497986,  8.95544812])

In [157]:
nas

{'auctioneerID': 2.0, 'MachineHoursCurrentMeter': 0.0}

We now have something we can pass to a random forest!

<a id='bottom'></a>

[Back to top](#plan)